# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = '/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [6]:
df_song_data = pd.read_json(filepath, lines='true')
df_song_data

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [7]:
#temp = pd.DataFrame()
#
#for file in song_files:
#    temp = pd.read_json(file, lines='true')
#    df_song_data = df_song_data.append(temp, ignore_index = True)
#df_song_data

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df_song_data[['song_id',
                          'title',
                          'artist_id',
                          'year',
                          'duration']].values[0].tolist()

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df_song_data[['artist_id',
                            'artist_name',
                            'artist_location',
                            'artist_latitude',
                            'artist_longitude']].values[0].tolist()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files('data/log_data')

In [13]:
filepath = '/home/workspace/data/log_data/2018/11/2018-11-30-events.json'

In [14]:
df_log_data = pd.read_json(filepath, lines='true')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df_time_nextsong = df_log_data[df_log_data.page == 'NextSong']

In [16]:
t = pd.to_datetime(df_time_nextsong.ts, unit='ms')

In [17]:
#start_time, hour, day, week, month, year, weekday
time_data = t,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.dayofweek
column_labels = ('timestamp','hour','day','week','month','year','day_of_week')

In [18]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
user_df = df_time_nextsong[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [25]:
df_artists_songs = df_time_nextsong[['artist','song','length','ts', 'userId', 'level', 'sessionId','location','userAgent']]
df_artists_songs.head(5)

,artist,song,length,ts,userId,level,sessionId,location,userAgent
0,Stephen Lynch,Jim Henson's Dead,182.85669,1543537327796,91,free,829,"Dallas-Fort Worth-Arlington, TX",Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...
1,Manowar,Shell Shock,247.56200,1543540121796,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,Morcheeba,Women Lose Weight (Feat: Slick Rick),257.41016,1543540368796,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,Maroon 5,Won't Go Home Without You,231.23546,1543540625796,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
4,Train,Hey_ Soul Sister,216.76363,1543540856796,73,paid,1049,"Tampa-St. Petersburg-Clearwater, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [26]:
for index, row in df_artists_songs.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    #songplay_data = (row.length,
    #                 row.userId,
    #                 row.level,
    #                 songid,
    
    #                 artistid,
    #                 row.sessionId,
    #                 row.location,
    #                 row.userAgent)
    
    #cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()
    print(row.song, row.artist, row.length, songid, artistid)

Jim Henson's Dead Stephen Lynch 182.85669 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Shell Shock Manowar 247.562 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Women Lose Weight (Feat: Slick Rick) Morcheeba 257.41016 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Won't Go Home Without You Maroon 5 231.23546 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Hey_ Soul Sister Train 216.76363 SOFSOCN12A8C143F5D ARXR32B1187FB57099
I'm In Miami Bitch LMFAO 227.99628 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Sexy Bitch DJ Dizzy 221.1522 SOFSOCN12A8C143F5D ARXR32B1187FB57099
The Cure & The Cause (Dennis Ferrer Remix) Fish Go Deep & Tracey K 377.41669 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Staring At Me M83 96.1824 SOFSOCN12A8C143F5D ARXR32B1187FB57099
The Second You Sleep Saybia 257.25342 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Wide Eyes Local Natives 266.05669 SOFSOCN12A8C143F5D ARXR32B1187FB57099
La Resistance (Medley) (LP Version) South Park 112.97914 SOFSOCN12A8C143F5D ARXR32B1187FB57099
Restless UNKLE Feat. Josh Homme 307.19955 S

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [23]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.